In [2]:
# from deepface import DeepFace
#
# embedding_objs = DeepFace.represent(
#   img_path = "./contacts_photos/TEST.jpg",
#   detector_backend = "retinaface",
#   model_name = "Facenet512",
#   align = 'true',
# )

24-11-23 00:34:05 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: C:\Users\minit\.deepface\weights\retinaface.h5
100%|██████████| 119M/119M [00:08<00:00, 13.7MB/s] 


In [1]:
# print(embedding_objs)
import tensorflow as tf
import keras
import deepface

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("DeepFace version:", deepface.__version__)

TensorFlow version: 2.9.1
Keras version: 2.9.0
DeepFace version: 0.0.93


In [9]:
from deepface import DeepFace

# Chemins des images
image1_path = "./contacts_photos/Jerem.jpg"
image2_path = "./contacts_photos/TEST2.jpg"

# Comparaison des images
result = DeepFace.verify(image1_path, image2_path, model_name='Facenet512', detector_backend='retinaface', align=True,
                         threshold=0.5)

# Résultat
print("Sont-elles la même personne ? :", result["verified"])
print("Distance :", result["distance"])
print("Détail :", result)

Sont-elles la même personne ? : True
Distance : 0.3291650681964142
Détail : {'verified': True, 'distance': 0.3291650681964142, 'threshold': 0.5, 'model': 'Facenet512', 'detector_backend': 'retinaface', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 777, 'y': 376, 'w': 202, 'h': 232, 'left_eye': (930, 455), 'right_eye': (836, 460)}, 'img2': {'x': 526, 'y': 359, 'w': 213, 'h': 259, 'left_eye': (690, 446), 'right_eye': (590, 447)}}, 'time': 7.39}


In [10]:
from deepface import DeepFace

# Chemins des images
image1_path = "./CelebrityFacesDataset/Angelina Jolie/001_fe3347c0.jpg"
image2_path = "./CelebrityFacesDataset/Angelina Jolie/016_8945d6ca.jpg"

# Comparaison des images
result = DeepFace.verify(image1_path, image2_path, model_name='Facenet512', detector_backend='retinaface', align=True,
                         threshold=0.5)

# Résultat
print("Sont-elles la même personne ? :", result["verified"])
print("Distance :", result["distance"])
print("Détail :", result)

Sont-elles la même personne ? : True
Distance : 0.1506341861457421
Détail : {'verified': True, 'distance': 0.1506341861457421, 'threshold': 0.5, 'model': 'Facenet512', 'detector_backend': 'retinaface', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 113, 'y': 98, 'w': 218, 'h': 307, 'left_eye': (265, 236), 'right_eye': (165, 218)}, 'img2': {'x': 129, 'y': 84, 'w': 184, 'h': 263, 'left_eye': (251, 195), 'right_eye': (166, 191)}}, 'time': 4.99}


In [11]:
import os
from itertools import combinations
from deepface import DeepFace

# Chemin vers le dataset
dataset_path = "./CelebrityFacesDataset"

# Paramètres du modèle
model_name = 'Facenet512'
detector_backend = 'retinaface'
threshold = 0.5

# Variables pour les résultats
true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0

# Liste des personnes (sous-dossiers)
persons = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Comparaison intra-personne (mêmes images)
for person in persons:
    person_folder = os.path.join(dataset_path, person)
    images = [os.path.join(person_folder, f) for f in os.listdir(person_folder) if f.endswith(('.jpg', '.png'))]

    # Crée des paires d'images dans le même dossier
    for img1, img2 in combinations(images, 2):
        result = DeepFace.verify(img1, img2, model_name=model_name, detector_backend=detector_backend, align=True,
                                 threshold=threshold)
        if result["verified"]:
            true_positives += 1
        else:
            false_negatives += 1

# Comparaison inter-personnes (images différentes)
for person1, person2 in combinations(persons, 2):
    person1_folder = os.path.join(dataset_path, person1)
    person2_folder = os.path.join(dataset_path, person2)

    images1 = [os.path.join(person1_folder, f) for f in os.listdir(person1_folder) if f.endswith(('.jpg', '.png'))]
    images2 = [os.path.join(person2_folder, f) for f in os.listdir(person2_folder) if f.endswith(('.jpg', '.png'))]

    # Compare une image de person1 avec une image de person2
    for img1 in images1:
        for img2 in images2:
            result = DeepFace.verify(img1, img2, model_name=model_name, detector_backend=detector_backend, align=True,
                                     threshold=threshold)
            if result["verified"]:
                false_positives += 1
            else:
                true_negatives += 1

# Calcul des métriques
total_positives = true_positives + false_negatives
total_negatives = true_negatives + false_positives

accuracy = (true_positives + true_negatives) / (total_positives + total_negatives)
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Résultats
print("Résultats du test :")
print(f"True Positives : {true_positives}")
print(f"False Positives : {false_positives}")
print(f"True Negatives : {true_negatives}")
print(f"False Negatives : {false_negatives}")
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision : {precision:.2f}")
print(f"Recall : {recall:.2f}")
print(f"F1 Score : {f1_score:.2f}")


KeyboardInterrupt: 

In [4]:
from deepface import DeepFace

# Générer les embeddings pour tous les visages présents dans l'image
embedding_objs = DeepFace.represent(
    img_path="./contacts_photos/JeremClement.jpg",
    detector_backend="retinaface",  # Détecteur pour identifier les visages
    model_name="Facenet512",  # Modèle utilisé pour les embeddings
    align=True,  # Aligner les visages détectés
    enforce_detection=True,  # Assure que les visages sont détectés
)

# Affichage des résultats
for i, embedding in enumerate(embedding_objs):
    print(f"Visage {i + 1}: {embedding}")


Visage 1: {'embedding': [-1.5118082761764526, 1.4601365327835083, -0.5694462060928345, -0.46456748247146606, -0.3929569721221924, -0.9432506561279297, 1.4444724321365356, 0.776737630367279, 1.019213080406189, -0.7817790508270264, 0.816615641117096, 0.34317418932914734, -1.036889910697937, -0.6223834753036499, 0.9665536880493164, -0.020830944180488586, -0.47937309741973877, -1.0261696577072144, -0.5008929967880249, -0.13527940213680267, 0.6537094712257385, -0.09408757090568542, -0.639856219291687, 0.0018600132316350937, 0.5863009095191956, 0.04498414695262909, -0.607003390789032, -0.7593380808830261, -1.0506372451782227, 0.6238212585449219, -1.1157262325286865, 0.3972708284854889, -1.0261425971984863, 0.5179714560508728, 0.35444730520248413, 1.9296705722808838, 1.1113674640655518, 0.09338071942329407, 0.3100767433643341, 0.37835779786109924, 0.32240238785743713, -0.346431165933609, -0.6923815011978149, 0.14607968926429749, -0.48382893204689026, -0.7985050082206726, 0.4852335453033447, 0

In [5]:
from deepface import DeepFace

# Générer les embeddings pour tous les visages présents dans l'image
embedding_objs = DeepFace.represent(
    img_path="./contacts_photos/Clem.jpg",
    detector_backend="retinaface",  # Détecteur pour identifier les visages
    model_name="Facenet512",  # Modèle utilisé pour les embeddings
    align=True,  # Aligner les visages détectés
    enforce_detection=True,  # Assure que les visages sont détectés
)

# Affichage des résultats
for i, embedding in enumerate(embedding_objs):
    print(f"Visage {i + 1}: {embedding}")


Visage 1: {'embedding': [0.29262542724609375, 1.901214361190796, 0.5536031126976013, -1.2647889852523804, 0.32825013995170593, 0.339370459318161, -1.3463554382324219, -0.33755582571029663, 1.2187129259109497, 1.3638293743133545, -0.43192440271377563, 0.2594631314277649, -1.5093276500701904, 0.6917766332626343, -1.3275011777877808, 1.1258808374404907, -0.9944764971733093, -0.7299253940582275, -0.3665893077850342, 0.9629489779472351, 0.08593473583459854, -0.8045029640197754, -0.4069877862930298, -0.3330029249191284, 0.6567071080207825, -0.44735026359558105, -0.5217258334159851, 1.3178366422653198, 0.9279975295066833, -0.7335941195487976, 0.2204117774963379, -1.34384024143219, 0.8801181316375732, 0.0036588292568922043, 1.2982286214828491, 1.887413740158081, 2.067654848098755, -0.4918787181377411, 1.2148551940917969, 0.5723613500595093, 0.20490606129169464, 0.19016790390014648, 0.6732357740402222, 1.0429317951202393, -0.6480908393859863, -0.6294004321098328, -1.2390626668930054, 1.00215947

In [6]:
from deepface import DeepFace

# Générer les embeddings pour tous les visages présents dans l'image
embedding_objs = DeepFace.represent(
    img_path="./contacts_photos/Jerem.jpg",
    detector_backend="retinaface",  # Détecteur pour identifier les visages
    model_name="Facenet512",  # Modèle utilisé pour les embeddings
    align=True,  # Aligner les visages détectés
    enforce_detection=True,  # Assure que les visages sont détectés
)

# Affichage des résultats
for i, embedding in enumerate(embedding_objs):
    print(f"Visage {i + 1}: {embedding}")


Visage 1: {'embedding': [-0.0923929437994957, -0.09239009022712708, -0.3988529145717621, -0.0814400166273117, -1.4772683382034302, -0.17579182982444763, 1.029139757156372, 1.5911258459091187, 1.2380977869033813, -1.9103987216949463, -0.10973934084177017, 0.8601358532905579, 0.13746453821659088, 1.265214443206787, -0.24435670673847198, 0.16681386530399323, -0.12922075390815735, 0.5063185095787048, -0.31164151430130005, -0.7499434351921082, -0.3565652370452881, 0.04371902346611023, 1.5574018955230713, 0.2019539177417755, 0.9795171618461609, -0.2331252247095108, -1.5725489854812622, -0.6581977605819702, -0.037036530673503876, -0.05550382658839226, -0.3092838227748871, 0.017958102747797966, -0.588019073009491, -0.3665885627269745, 0.7948889136314392, 2.9618005752563477, 0.02744627371430397, 0.6667462587356567, -0.4062204360961914, 0.6210648417472839, 1.0197731256484985, 0.22241151332855225, -0.41544294357299805, -0.04946672171354294, -0.3628058433532715, -1.035767912864685, -0.751827299594

# Qdrant

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(url="http://localhost:6333")
client.create_collection(
    collection_name="Image",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `Profile` already exists!"},"time":0.000065542}'

In [15]:
from deepface import DeepFace
from qdrant_client.models import PointStruct
from qdrant_client import QdrantClient

# Initialisation du client Qdrant
client = QdrantClient(url="http://localhost:6333")


def generate_embeddings(img_path: str, model_name: str = "Facenet512", detector_backend: str = "retinaface"):
    """
    Génère les embeddings à partir d'une image contenant un ou plusieurs visages.

    Args:
        img_path (str): Chemin vers l'image.
        model_name (str): Modèle utilisé pour les embeddings (par défaut : "Facenet512").
        detector_backend (str): Backend utilisé pour détecter les visages (par défaut : "retinaface").

    Returns:
        list: Liste des embeddings générés pour chaque visage détecté.
    """
    try:
        embedding_objs = DeepFace.represent(
            img_path=img_path,
            model_name=model_name,
            detector_backend=detector_backend,
            align=True,
            enforce_detection=True
        )
        return [embedding["embedding"] for embedding in embedding_objs]
    except Exception as e:
        print(f"Erreur lors de la génération des embeddings pour {img_path} : {e}")
        return []


def add_profile_to_qdrant(collection_name: str, embedding: list, profile_id: int, name: str, surname: str):
    """
    Ajoute un profil à une collection Qdrant.

    Args:
        collection_name (str): Nom de la collection.
        embedding (list): Vecteur d'embedding du profil.
        profile_id (int): Identifiant unique du profil.
        name (str): Prénom de l'utilisateur.
        surname (str): Nom de famille de l'utilisateur.
    """
    try:
        operation_info = client.upsert(
            collection_name=collection_name,
            wait=True,
            points=[
                PointStruct(
                    id=profile_id,
                    vector=embedding,
                    payload={
                        "name": name,
                        "surname": surname
                    },
                ),
            ]
        )
        print(f"Profil ajouté avec succès : {name} {surname} (ID: {profile_id})")
        return operation_info
    except Exception as e:
        print(f"Erreur lors de l'ajout du profil {name} {surname} : {e}")


# Liste des profils avec chemins vers les photos
profiles = [
    {
        "id": 1,
        "name": "Clément",
        "surname": "Tableau",
        "img_path": "./contacts_photos/Clem.jpg"
    },
    {
        "id": 2,
        "name": "Jérémy",
        "surname": "Billuart",
        "img_path": "./contacts_photos/Jerem.jpg"
    }, {
        "id": 3,
        "name": "Mathis",
        "surname": "Champagne",
        "img_path": "./contacts_photos/Mathis.jpg"
    }, {
        "id": 4,
        "name": "Romeo",
        "surname": "Correc",
        "img_path": "./contacts_photos/Romeo.jpg"
    }
]

# Ajout des profils à Qdrant après génération des embeddings
collection_name = "Profile"
for profile in profiles:
    embeddings = generate_embeddings(profile["img_path"])
    if embeddings:
        for i, embedding in enumerate(embeddings):
            add_profile_to_qdrant(
                collection_name=collection_name,
                embedding=embedding,
                profile_id=int(f"{profile['id']}{i}"),  # ID unique pour chaque visage
                name=profile["name"],
                surname=profile["surname"]
            )
    else:
        print(f"Pas d'embeddings générés pour {profile['name']} {profile['surname']}")

Profil ajouté avec succès : Clément Tableau (ID: 10)
Profil ajouté avec succès : Jérémy Billuart (ID: 20)
Profil ajouté avec succès : Mathis Champagne (ID: 30)
Profil ajouté avec succès : Romeo Correc (ID: 40)


In [18]:
from deepface import DeepFace
from qdrant_client.models import Filter
from qdrant_client import QdrantClient

# Initialisation du client Qdrant
client = QdrantClient(url="http://localhost:6333")


def generate_embeddings_from_image(img_path: str, model_name: str = "Facenet512", detector_backend: str = "retinaface"):
    """
    Génère les embeddings pour tous les visages dans une image.

    Args:
        img_path (str): Chemin vers l'image.
        model_name (str): Modèle utilisé pour les embeddings.
        detector_backend (str): Backend pour la détection des visages.

    Returns:
        list: Liste des vecteurs d'embeddings.
    """
    try:
        embedding_objs = DeepFace.represent(
            img_path=img_path,
            model_name=model_name,
            detector_backend=detector_backend,
            align=True,
            enforce_detection=True
        )
        return [embedding["embedding"] for embedding in embedding_objs]
    except Exception as e:
        print(f"Erreur lors de la génération des embeddings pour {img_path} : {e}")
        return []


def query_similar_profiles(embedding: list, collection_name: str, limit: int = 3, threshold: float = 0.5):
    """
    Requête dans Qdrant pour trouver les profils les plus similaires à un embedding donné.

    Args:
        embedding (list): L'embedding pour lequel rechercher des similitudes.
        collection_name (str): Nom de la collection dans Qdrant.
        limit (int): Nombre maximum de résultats à retourner.
        threshold (float): Seuil minimum de similarité.

    Returns:
        list: Résultats des points similaires au-dessus du seuil.
    """
    try:
        hits = client.search(
            collection_name=collection_name,
            query_vector=embedding,
            limit=limit,
        )
        # Filtrer les résultats en fonction du seuil
        filtered_hits = [hit for hit in hits if hit.score >= threshold]
        return filtered_hits
    except Exception as e:
        print(f"Erreur lors de la requête de recherche : {e}")
        return []


# Générer les embeddings pour l'image contenant potentiellement plusieurs visages
img_path = "./contacts_photos/JeremLoin.PNG"
embeddings = generate_embeddings_from_image(img_path)

if embeddings:
    # Pour chaque embedding détecté dans l'image
    for i, embedding in enumerate(embeddings):
        print(f"\nRecherche pour le visage {i + 1}:")
        results = query_similar_profiles(embedding, collection_name="Profile", threshold=0.5)
        if results:
            for result in results:
                print(f"Profil similaire trouvé : {result.payload}, score : {result.score}")
        else:
            print("Aucun résultat au-dessus du seuil.")
else:
    print("Aucun visage détecté ou erreur lors de la génération des embeddings.")


Recherche pour le visage 1:
Profil similaire trouvé : {'name': 'Jérémy', 'surname': 'Billuart'}, score : 0.632906
